<img src="EEGRIDAnalytics.png">

In [1]:
import pandas as pd

import ipywidgets as widgets
import json

import datetime as dt
import numpy as np
from ipyleaflet import *
from ipywidgets import *


In [2]:
ee_basemaps={}
ee_basemaps['ARCGIS_Satelite'] = TileLayer(url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}')

# Loops through all ipyleaflet basemaps
for item in basemaps.values():
    try:
        name = item['name']
        basemap = 'basemaps.{}'.format(name)
        ee_basemaps[name] = basemap_to_tiles(eval(basemap))
    except:
        for sub_item in item:
            name = item[sub_item]['name']
            basemap = 'basemaps.{}'.format(name)
            basemap = basemap.replace('Mids', 'Modis')
            ee_basemaps[name] = basemap_to_tiles(eval(basemap))
list_maps = list(ee_basemaps.keys())
list_maps_2 = list_maps[0:2]

In [3]:
def change_map_layer(change):
    print('layer change start')
    basemap_name = change['new']
    list_classes1 = [m.layers[jj].__class__.__name__ for jj in range(0,len(m.layers))]
    if len(list_classes1)==1:
        m.add_layer(ee_basemaps[basemap_name])
    else:
        list_classes2 = [m.layers[jj].__class__.__name__ for jj in range(1,len(m.layers))]
        if 'TileLayer' in list_classes2:
            old_basemap1 = m.layers[list_classes2.index('TileLayer')+1]
            m.substitute_layer(old_basemap1, ee_basemaps[basemap_name])
        else:
            m.add_layer(ee_basemaps[basemap_name])
            

In [41]:
FV_df1 = pd.read_csv('Dummy_Nodes.csv')

In [42]:
FV_df1

,BusNumber,BusName,kVLevel,LoadZone,TSPName,Latitude,Longitude
0,900000,Bus_1,23,Zone_1,TSP_1,30.59791,-95.90799
1,900001,Bus_2,23,Zone_1,TSP_1,30.74161,-96.02626
2,900002,Bus_3,23,Zone_1,TSP_1,30.74661,-96.01626
3,900003,Bus_4,23,Zone_1,TSP_1,30.46050,-96.09110
4,900004,Bus_5,23,Zone_1,TSP_1,30.30316,-96.27729
...,...,...,...,...,...,...,...
94,900094,Bus_95,23,Zone_1,TSP_1,32.77310,-95.80626
95,900095,Bus_96,23,Zone_1,TSP_1,32.77581,-95.79506
96,900096,Bus_97,23,Zone_1,TSP_1,32.78081,-95.78506
97,900097,Bus_98,23,Zone_1,TSP_1,32.52895,-95.30106


In [43]:
FV_df1['BusNo&Name'] = FV_df1['BusNumber'].astype('str').str.cat(FV_df1['BusName'], sep= '-')
Bus_no_list =sorted(FV_df1["BusNo&Name"].unique())

In [44]:
def Bus_No_control(change):
    if change.new != None:
        flag = 1
        df88_1=FV_df1[FV_df1['BusNo&Name'] == '900007-Bus_8']
        print(df88_1)
        x = df88_1['Latitude']
        y = df88_1['Longitude']
        bus_fp = df88_1['BusNumber'].iloc[0] 
        marker = Marker(location = (x.iloc[0],y.iloc[0]),draggable=False)

        message2 = HTML()
        message2.value ="Bus No.:"+str(df88_1['BusNumber'].iloc[0])+'<br>'+"BusName:"+df88_1['BusName'].iloc[0] \
                                +'<br>'+'KV:'+str(df88_1['kVLevel'].iloc[0])+'<br>'+'Load Zone:'+df88_1['LoadZone'].iloc[0]\
                                +'<br>'+'TSP Name:'+df88_1['TSPName'].iloc[0]
        marker.popup = message2


        m.add_layer(marker)
        m.center = (x.iloc[0],y.iloc[0])
        m.zoom = 18


In [45]:
center=(31.432,-102.4377)
#print(center)
m = Map(center=center, zoom_control=False, zoom=5,inertia = True, max_zoom = 30,scroll_wheel_zoom = True, inertia_max_speed = 1500, preferCanvas = True)
m.layout.height='500px'

full_screen = FullScreenControl(position='bottomleft')
m.add_control(full_screen)
m.add_control(ZoomControl(position='bottomleft'))

dd1 = widgets.Dropdown(options=Bus_no_list,value=None,description='Bus No.&Name:',layout=Layout(width='250px'))
dd1.observe(Bus_No_control, names="value")
widget_control2 = WidgetControl(widget=dd1, position='topleft')
m.add_control(widget_control2)


Map_layer = widgets.Dropdown(options=list_maps_2,value=None,description='BaseMap:')
Map_layer.observe(change_map_layer, 'value')
base_layer = WidgetControl(widget=Map_layer, position='topright')
m.add_control(base_layer)

m

Map(center=[31.432, -102.4377], controls=(AttributionControl(options=['position', 'prefix'], position='bottomr…

In [46]:
with open('Dummy_Nodes.geojson', 'r') as f:
    data = json.load(f)
features = data['features']
marker_c = []
for i in range(len(features)):
    location=(features[i]['geometry']['coordinates'][1],features[i]['geometry']['coordinates'][0])
    Bus_Number = features[i]['properties']['Bus Number']
#     print(PSSE_New_BUS)
#     PSSE_New_BUS = str(PSSE_New_BUS)
    Bus_Name = features[i]['properties']['Bus Name']

    kV_Level = features[i]['properties']['kV Level']
    Load_Zone = features[i]['properties']['Load Zone']
#     County = str(County)

    marker = Marker(location=location,draggable=False)
    message2 = HTML()
    message2.value ="Bus_Number: "+str(Bus_Number)+'<br>'+"Bus_Name "+str(Bus_Name)\
                    +'<br>'+'kV_Level: '+str(kV_Level)+'<br>'+'Load_Zone: '+str(Load_Zone)
    
    marker.icon = AwesomeIcon(name='battery',marker_color='red',icon_color='black', spin=False)

    marker.popup = message2
    marker_c.append(marker)
marker_cluster = MarkerCluster(markers =marker_c)   
m.add_layer(marker_cluster)